## Parameter sets
Change, create and use user-defined parameter sets

In [ ]:
!pip install pybamm
import pybamm as pb

In [ ]:
# parameter values are held in a dictionary, which can be looked up via:
parameter_values = pb.ParameterValues("Marquis2019")
print(parameter_values["Upper voltage cut-off [V]"])
print(parameter_values["Lower voltage cut-off [V]"])

# initialise the model
model = pb.lithium_ion.DFN()

In [ ]:
experiment3 = pb.Experiment(
    # Initialize
    [
        "Hold at 4.2 V until C/100",
        "Rest for 4 hours",
    ]
    # Capacity check
    + [(
        "Discharge at C/10 until 2.5 V",
        "Charge at C/10 until 4.2 V",
        "Hold at 4.2 V until C/100"
    )]
    # Ageing cycles
    + [(
        "Discharge at 1C until 2.5 V",
        "Charge at 0.3C until 4.2 V",
        "Hold at 4.2 V until C/100",
    )] * 10
    # Capacity check
    + [(
        "Discharge at C/10 until 2.5 V",
        "Charge at C/10 until 4.2 V",
        "Hold at 4.2 V until C/100"
    )]
)

# attempt simulation using this set
simulation3 = pb.Simulation(model, experiment=experiment3)
solution = simulation3.solve([0, 3600])
simulation3.plot()

In [ ]:
# attempt simulation using "OKane2022" parameter set
parameter_values = "OKane2022" 
simulation3 = pb.Simulation(
    model,
    experiment=experiment3,
    parameter_values=parameter_values
)
simulation3.plot()

In [ ]:
# change/update the parameter set for a more significant degradation
parameter_values.update({
    "Outer SEI solvent diffusivity [m2.s-1]": 1.25e-20,
    "Lithium plating kinetic rate constant [m.s-1]": 1e-8,
    "Dead lithium decay constant [s-1]": 4e-6,
    "Negative electrode cracking rate": 1.95e-18,
    "Negative electrode LAM constant proportional term [s-1]": 5.5556e-6,
    "Positive electrode LAM constant proportional term [s-1]": 5.5556e-6,
})

simulation3 = pb.Simulation(
    model,
    experiment=experiment3,
    parameter_values=parameter_values
)
simulation3.plot()

You can create and import your own parameter sets:

```python
from your_parameter_set import get_parameter_values
parameter_values = get_parameter_values()
```

## Input parameters

This is useful for wanting to see how one or a small subset of parameter affects the solution. Instead of recreating a "simulation" object, use an "input parameter":

In [ ]:
parameter_values = pb.ParameterValues("Marquis2019")
parameter_values.update({
    "Current function [A]": "[input]",
})

# allows input of a parameter when the model gets solved
simulation = pb.Simulation(
    model,
    parameter_values=parameter_values
)
solution = simulation.solve([0, 3600], inputs={"Current function [A]": 2})